<a href="https://colab.research.google.com/github/AlexFly666/LLM-in-Practice/blob/main/chapter09/0_quikstart/firestore_chat_history_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 依赖库安装命令-最新版本
# !pip install google-cloud-firestore \
#     langchain-google-firestore \
#     langchain-openai \
#     openai
# 依赖库安装命令-langchain 0.3
!pip install google-cloud-firestore==2.20.0 \
    langchain-google-firestore==0.5.0 \
    langchain-openai==0.3.4 \
    openai==1.61.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33


In [10]:
!pip show google-cloud-firestore \
    langchain-google-firestore \
    langchain-openai \
    openai

Name: google-cloud-firestore
Version: 2.20.0
Summary: Google Cloud Firestore API client library
Home-page: https://github.com/googleapis/python-firestore
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: google-api-core, google-auth, google-cloud-core, proto-plus, protobuf
Required-by: firebase-admin, langchain-google-firestore
---
Name: langchain-google-firestore
Version: 0.5.0
Summary: LangChain integrations for Google Cloud Firestore
Home-page: https://github.com/googleapis/langchain-google-firestore-python
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
                              Apache License
                        Version 2.0, January 2004
                     http://www.apache.org/licenses/

TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

1. Definitions.

   "License" shall mean the terms and conditions for use, reproduction,
   and distri

In [2]:
# 导入 Firestore 客户端库
from google.cloud import firestore
# 导入 Langchain Firestore Chat 消息历史记录类
from langchain_google_firestore import FirestoreChatMessageHistory
# 导入 Langchain OpenAI Chat 模型
from langchain_openai import ChatOpenAI

In [ ]:
"""
步骤来复现这个例子:
1. 创建一个 Firebase 账户
2. 创建一个新的 Firebase 项目
    - 复制项目 ID
3. 在 Firebase 项目中创建一个 Firestore 数据库
4. 在你的电脑上安装 Google Cloud CLI
    - https://cloud.google.com/sdk/docs/install
    - 使用你的 Google 账户验证 Google Cloud CLI
        - https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev
    - 将你的默认项目设置为你创建的新 Firebase 项目
5. 在 Google Cloud 控制台中启用 Firestore API:
    - https://console.cloud.google.com/apis/enableflow?apiid=firestore.googleapis.com&project=crewai-automation
"""

In [11]:
#向Google Cloud进行身份验证，以访问您的Google Cloud项目。
# 如果您使用 Colab 运行此笔记本，请使用下面的单元格并继续。
from google.colab import auth
auth.authenticate_user()
# 其他方式参考
# https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env

In [9]:
# 设置 Firebase Firestore 项目 ID
PROJECT_ID = "llminpractice"
# 设置会话 ID, 用于区分不同的用户或会话，这里使用 "user_session_new" 作为示例
SESSION_ID = "user_session_new"  # This could be a username or a unique ID
# 设置 Firestore 集合名称，用于存储聊天历史记录，这里使用 "chat_history" 作为示例
COLLECTION_NAME = "chat_history"

# 初始化 Firestore 客户端
print("Initializing Firestore Client...")
# 创建 Firestore 客户端对象，需要传入项目 ID
client = firestore.Client(project=PROJECT_ID)

# 初始化 Firestore Chat 消息历史记录
print("Initializing Firestore Chat Message History...")
# 创建 FirestoreChatMessageHistory 对象，用于管理聊天历史记录的存储和检索
chat_history = FirestoreChatMessageHistory(
    session_id=SESSION_ID, # 传入会话 ID，用于区分不同会话的数据
    collection=COLLECTION_NAME, # 传入集合名称，指定数据存储在哪个集合中
    client=client, # 传入 Firestore 客户端对象，用于与 Firestore 数据库交互
)
print("Chat History Initialized.")
# 打印当前的聊天历史记录，刚初始化时应该是空的
print("Current Chat History:", chat_history.messages)

# 初始化 Chat 模型
# 智谱API调用(glm-4v-flash)
model = ChatOpenAI(
    api_key="XXX",
    base_url="https://open.bigmodel.cn/api/paas/v4/",
    model="glm-4v-flash"
)

print("Start chatting with the AI. Type 'exit' to quit.")
# 进入聊天循环
while True:
    # 获取用户输入
    human_input = input("User: ")
    # 如果用户输入 "exit"，则退出循环
    if human_input.lower() == "exit":
        break

    # 将用户消息添加到聊天历史记录中
    chat_history.add_user_message(human_input)

    # 使用 Chat 模型处理聊天历史记录，并获取 AI 的回复
    ai_response = model.invoke(chat_history.messages)
    # 将 AI 的回复添加到聊天历史记录中
    chat_history.add_ai_message(ai_response.content)

    # 打印 AI 的回复
    print(f"AI: {ai_response.content}")

Initializing Firestore Client...
Initializing Firestore Chat Message History...
Chat History Initialized.
Current Chat History: [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), HumanMessage(content='java', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you with Java?', additional_kwargs={}, response_metadata={}), HumanMessage(content='使用java实现一个快速排序', additional_kwargs={}, response_metadata={}), AIMessage(content='当然可以！下面是一个简单的Java快速排序示例：\n\n```java\npublic class QuickSort {\n    public static void main(String[] args) {\n        int[] arr = {9, 7, 5, 11, 12, 2, 14, 3, 10, 6};\n        quickSort(arr, 0, arr.length - 1);\n        for (int i : arr) {\n            System.out.print(i + " ");\n        }\n    }\n\n    public static void quickSort(int[] arr, int left, int right) {\n        if (left < right) {\n            // 找到基准值的位置\n            int pivotIndex = partition(arr, left, right);\n\n            // 对基准值的左边进行递归排序\n  